<a href="https://colab.research.google.com/github/vishwajeet-hogale/IPO-rep/blob/main/TestBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

In [ ]:
!pip install ./transformers
!pip install tensorboardX


In [ ]:
!mkdir dataset \
&& cd dataset \
&&   \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
# !export SQUAD_DIR=/content/dataset \
# && python transformers/examples/run_squad.py \
#   --model_type albert \
#   --model_name_or_path albert-base-v2 \
#   --do_train \
#   --do_eval \
#   --do_lower_case \
#   --train_file $SQUAD_DIR/train-v2.0.json \
#   --predict_file $SQUAD_DIR/dev-v2.0.json \
#   --per_gpu_train_batch_size 12 \
#   --learning_rate 3e-5 \
#   --num_train_epochs 1.0 \
#   --max_seq_length 384 \
#   --doc_stride 128 \
#   --output_dir /content/model_output \
#   --save_steps 1000 \
#   --threads 4 \
#   --version_2_with_negative 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
print(torch.__version__)


In [ ]:

from transformers import (
    

    BertTokenizer,
    BertForQuestionAnswering,
    squad_convert_examples_to_features
)


from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model



tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [ ]:
#we have to read csv here!
context = """ (Bloomberg) --

The U.S. sent 2.5 million doses of Moderna Inc.’s vaccine to Taiwan, reflecting part of President Joe Biden’s pledge to donate 25 million shots worldwide to stem the pandemic. Amid a spike in cases over the past six weeks, Taiwan still faces a serious shortage of vaccines.

India may be hit by a third wave of Covid-19 far sooner than predicted as people are ignoring guidelines, Randeep Guleria, director at the state-run All India Institute of Medical Sciences, told the Times of India. Infections could start rising again in 12 to 16 weeks, he said.

Brazil reached 500,000 deaths from Covid-19 on Saturday, a toll second only to the U.S. As the milestone was officially passed, thousands protested in major cities against President Jair Bolsonaro and his handling of the pandemic.

Key Developments:

Global Tracker: Cases exceed 178 million; deaths pass 3.8 millionVaccine Tracker: More than 2.59 billion doses administeredEpidemiologists are starting to shift focus away from Covid case countsMany Brazilians insist on Pfizer even with 500,000 dead of CovidCovid counts hit zero in U.S. hospitals once overrun by victimsU.K. is a test case for Covid endgame as variant upends the mathSubscribe to a daily update from Bloomberg’s Prognosis team here.

Hong Kong to Examine Easing Distancing Rules (1:10 p.m. HK)

Hong Kong’s government will “carefully examine” if there’s room to relax social distancing measures with current rules set to expire this week, Chief Secretary for Administration Matthew Cheung said in a blog post on Sunday. Hong Kong has reported zero locally-transmitted cases for more than 10 days, with the current social distancing rules set to remain until June 23.

Ugandan Olympic Team Member Tests Positive on Arrival (1 p.m. HK)

A member of Uganda’s Olympic team tested positive for Covid-19 on arrival in Japan, Kyodo News reported, in the first such case ahead of the Games, which begin in just over 30 days.

The team member was one of nine competitors arriving for the Olympics, and had received two doses of AstraZeneca’s vaccine, the report said. The athlete was tested on arrival at Narita airport and was denied entry to the country after the positive result.

Story continues

Sydney Issues Selective Public Restrictions After Mall Spread (11:30 a.m. HK)

Compulsory indoor mask wearing may be mandated for residents of Australia’s most-populous city if the case count worsens by tomorrow, officials said Sunday, after a number of locals acquired the Delta strain of Covid-19 at one of the city’s busiest retail hubs.

New South Wales recorded two new locally acquired cases on Sunday and expanded its mandatory mask-wearing policy for people on public transport and any indoor activity except eating or drinking across a list of selected suburbs beyond Bondi and nearby areas where restrictions were already in place.

The patchwork policy that selects specific regions ranging from the city’s inner west to the southern coastal reach of Wollongong stops short of a more blanket call. “We’ve held off being more heavy handed than we are in relation to what we’re asking people to do, but that will depend on what cases, if any, emerge in the next few days,” New South Wales Premier Gladys Berejiklian told a news conference Sunday.

Olympic Opening Ceremony May Have 20,000 Spectators (11:30 a.m. HK)

As many as 20,000 people may be permitted to attend the opening ceremony of the Olympic Games, Japanese broadcaster NTV reported on Sunday, as organizers of the Games near a decision expected on Monday on whether the Games will be held with spectators. A cap of up to 10,000 people is likely at other events, according to reports.

Meanwhile, compeptitors at the Games will be able to bring alcoholic drinks into the athletes’ village, Kyodo reported. Residents won’t be able to drink in public or common areas, but can consume alcohol in their rooms.

Australia’s Queensland Eases Restrictions Further (8:45 a.m. HK)

Australia’s third-most populous state has announced further easing of public restrictions, including lifting limits on home gatherings, outdoor places and aged care and hospital visits.

The changes to the state’s road map to re-opening take effect next Friday. Under the changes, self-service food setups are also now allowed, in addition to spacing in venues of three people per four square meters up from one person per two square meters.

The state, which counts tourism as a key industry, is still closed to visitors from parts of Sydney and all of Melbourne, but the new rules will allow hotels to open more rooms in time for intrastate activity during school holidays beginning next week, Premier Annastacia Palaszczuk said in a statement.

Tokyo Scraps Olympic Public Viewing Sites (8 a.m. HK)

Tokyo Governor Yuriko Koike scrapped plans for several Olympic public viewing sites in the capital where events from the Games were to be screened.

Koike said she had canceled the plans after meeting with Prime Minister Yoshihide Suga to discuss the state of the pandemic, with Tokyo and several other areas set to end the state of emergency today. Some of the sites will be turned into mass vaccination centers instead.

L.A. Says Variants First Found in India on Rise (5:56 p.m. NY)

Los Angeles’s county health officials said they’ve seen an increase in variants first found in India in the past few weeks.

The health department started reviewing tests at labs with data on residents and noticed small numbers of the highly transmissible delta variant, along with a few cases of the related kappa variant, since early April.

Since then and through early June, 64 infections caused by the delta variant were detected among residents, with most of them identified in the past few weeks. The variant transmitted within households in more than half of the cases, it added. Delta now makes up 10% of cases in the U.S. and is expected to become the dominant strain, the Centers for Disease Control and Prevention says.

Need for Booster Shot Uncertain, WHO Expert Says (5:35 p.m. NY)

As some governments and pharmaceutical officials prepare for Covid booster shots targeting more-infectious virus variants, health authorities say it’s too early to tell if they will be required.

“We do not have the information that’s necessary to make the recommendation on whether or not a booster will be needed,” Soumya Swaminathan, the World Health Organization’s chief scientist, said in a Zoom interview Friday. The “science is still evolving.”

Such a call is “premature” while high-risk individuals in most of the world haven’t yet completed a first course of vaccination, Swaminathan said.

Brazil Passes 500,000 Fatalities (5:22 p.m. NY)

Brazil reached 500,000 deaths from Covid-19 on Saturday, a toll second only to the U.S. As the milestone was officially passed, thousands protested in major cities against President Jair Bolsonaro and his handling of the pandemic.

Demonstrators asked for the president’s impeachment, a faster vaccination pace and an increase in cash assistance during a third wave of Covid-19 in Latin America’s largest economy.

“I am working tirelessly to vaccinate all Brazilians in the shortest time possible and to change this scenario that has plagued us for over a year,” Health Minister Marcelo Queiroga tweeted on Saturday before the official toll was announced. Almost 30% of the nation has received at least one dose of vaccine, and 11.6% is fully vaccinated, according to the Bloomberg Vaccine Tracker.

Another 2,301 fatalities were reported Saturday for a total of 500,800, according to Health Ministry data.

Mexico City to Close Schools (2:35 p.m. NY)

Mexico City schools will suspend in-person classes as of June 21 after the local government moved to a yellow code alert from a green light according to the country’s stoplight system.

The federal education authority for Mexico City said that it is suspending classes including public and private elementary, high school, college-level and teacher training, according to a joint statement with the education ministry.

U.S. Sends Taiwan 2.5 Million Doses (12:22 p.m. NY)

The U.S. sent 2.5 million doses of Moderna Inc.’s vaccine to Taiwan, reflecting part of President Joe Biden’s pledge to donate 25 million shots worldwide to stem the pandemic.

While the U.S. has criticized China for what it says have been attempts to block vaccine supplies to Taiwan, an administration official said the U.S. shipment came with no political strings attached.

Taiwan is receiving Moderna shots on a plane that left Memphis on Saturday morning, according to two administration officials.

Malawi Runs Out of Vaccines (11:53 a.m. NY)

Malawi has run out of vaccines because of delays in shipping, its health ministry said on Saturday, according to Agence France Presse. The nation’s inoculation campaign began in April, and many people are due for their second dose of the AstraZeneca vaccine.

With a new viral wave sweeping across Africa, Gambia said on Thursday that it too had run out of its AstraZeneca shots. It didn’t say when new doses will be imported.

U.S. Outbreak Improves as Variant Threatens (8:01 a.m. NY)

The U.S. reported just over 13,000 new cases on Friday as new infections dropped to the level seen in the first lockdown in March 2020, according to data compiled by Johns Hopkins University and Bloomberg. Friday’s number was a 10th that of the same day six months ago.

Another 368 deaths were reported, with average daily deaths about half of a month ago.

The continued improvement is clouded by the spread of the highly transmissible delta variant first found in India and hampering a final reopening in England. On Friday, President Biden called it “a serious and deadly threat” and urged Americans to get vaccinated. Full vaccination has been shown effective against the variant.

More stories like this are available on bloomberg.com

Subscribe now to stay ahead with the most trusted business news source.

©2021 Bloomberg L.P."""
questions = ["which company is going to be listed?","which company is going to be IPO?","which company is going to be public?","which company is going to be listed on stock exchange?","which company is about to launch initial public offering?","which company is about to launch IPO?","which company is it talking about?","where is the company located?","in which country is it?","What are the maximum shares offered?","What are the minimum shares offered?"]
# questions1 = ["What is the first trading date?","Which currency is being used?","What is the symbol of the company?","Number of shares offered","In which country is it?","What is the issuer name","Which stock exchange does it belong to?","What is the share price of each share","Which company share are they selling?"]
# Run method
predictions = run_prediction(questions, context)
print(predictions)
# Print results
import statistics as st
prelist = [i for i in predictions.values() if i != '']
try:
  print(st.mode(prelist))
except:
  print(prelist)